# Kryptografia asymetryczna - kryptosystem RSA 
Kryptografia asymetryczna charakteryzuje się wykorzystaniem **pary kluczy publiczny-prywatny** (stąd nazwa kryptografia z kluczem publicznym). Klucz publiczny może być swobodnie dystrybuowany otwartym kanałem i służy do szyfrowania (a także do weryfikowania podpisu). Klucz prywatny musi być utrzymywany w tajności i służy do deszyfrowania (lub tworzenia podpisu). 

Chronologicznie pierwszym kryptosystemem asymetrycznym był protokół wymiany kluczu Diffiego-Hellmana-Merkla. Służy on bezpiecznej wymiany danych, które mogą być wykorzystane jako tajne klucze kryptograficzne lub mogą być użyte do wyprodukowania kluczy. 

Najbardziej znanym kryptosystem z kluczem publicznym jest RSA (nazwa pochodzi od wynalazów: Rivest, Shamir i Adlemann). RSA umożliwia szyfrowanie danych jak również realizację podpisu cyfrowego. Bezpieczeństwo RSA opiera się na obliczeniowej trudności rozwiązania **problemu faktoryzacji liczb całkowitych złożonych**. 

### Algorytm szybkiego potęgowania 
1. Zwykłe potęgowanie $n^{exp}$: $exp$ mnożeń 
2. Algorytm szybkiego potęgowania: część mnożeń zastępujemy podnoszeniem do kwadratu (_squaring_).
    __Skąd mamy wiedzieć kiedy mnożyć, a kiedy potęgować?__

In [1]:
def fastModularExponentation(b, exp, m):
    res = 1
    while exp > 1:
        if exp & 1:
            res = (res * b) % m
        b = b ** 2 % m
        exp >>= 1
    return (b * res) % m

### Skąd wziąć dużą liczbę pierwszą? 
**Wylosować i sprawdzić czy jest pierwsza!**

Test probabilistyczny, np. Rabina-Millera. 

In [8]:
import random
def isPrime3(n, k):
    
    def millerTest(d, n):
        a = random.randint(2, n-2)
        x = fastModularExponentation(a,d, n)
        
        if x == 1 or x == n-1:
            return True
        
        while d != n-1:
            x = (x*x)%n
            d *= 2
            if x == 1: return False
            if x == n-1: return True
        return False
    
    if n < 2: return False
    if n < 4: return True
    if n % 2 == 0: return False    # speedup

    s = 0
    d = n-1
    while d % 2 == 0:
        s += 1
        d //= 2
        
    for i in range(k):
        if not millerTest(d,n):
            return False

    return True    

for i in range(65500,65600):            
    if(isPrime3(i,10)): print(i)

65519
65521
65537
65539
65543
65551
65557
65563
65579
65581
65587
65599


### Obliczanie elementu odwrotnego w ciele skończonym
1. Algorytm Euklidesa - znajdowanie największego wspólnego dzielnika
2. Rozszerzony Algorytm Euklidesa - obliczanie elementu odwrotnego względem mnożenia w ciele skończonym

Przeanalizuj poniższy kod.

In [3]:

## Funkcje pomocnicze 
def gcd(a, b):
    # GCD - Greatest Common Divisor, Największy wspólny dzielnik 
    while a != 0:
        a, b = b % a, a
    return b

# wersja rekurencyjna
def gcdExtendedRecursion(a, b):  
    # przypadek trywialny
    if a == 0 :   
        return b,1,0             
    gcd,x1,y1 = gcdExtendedRecursion(b%a, a) #rekurencyjny Euklides      
   
    # sumowanie powrotne  
    y = x1 - (b//a) * y1  
    x = y1  
    
    return gcd,x,y 

#wersja iteracyjna
def findModInverse(a, m):
    # Zwraca liczbę x odwrotną do a ciele skończonym modulo m 
    # czyli (a*x) % m =1 

    if gcd(a, m) != 1:
        return None #a i m muszą być względnie pierwsze aby istniał element odwrotny 

    # Rozszerzony algorytm Euklidesa 
    u1, u2, u3 = 1, 0, a
    v1, v2, v3 = 0, 1, m
    while v3 != 0:
        q = u3 // v3 # // operator dzielenie całkowitoliczbowego 
        v1, v2, v3, u1, u2, u3 = (u1 - q * v1), (u2 - q * v2), (u3 - q * v3), v1, v2, v3
    return u1 % m


a, b = 154,801
g, x, y = gcdExtendedRecursion(a, b)  
print("gcd(", a , "," , b, ") = ", g, x, y)
print("Element odwrotny do ", a, "modulo", b, ":", y%b)

print(findModInverse(154,801))

gcd( 154 , 801 ) =  1 5 -26
Element odwrotny do  154 modulo 801 : 775
775


## Generowanie kluczy w kryptosystemie RSA
Potrzebujemy dwóch liczb pierwszych o naprawdę dużych rozmiarach - 2048 bitów obecnie uważa się niezbyt bezpieczny wybór. 4096 bitów jest z kolei wielkością nieco kłopotliwą w użytkowaniu.
### Zadanie
1. Napisz funkcję generującą liczbę pierwszą o określonej długości w bitach. 

In [13]:
print(2**12)
print(isPrime3(
814208967992483828072695530334581251973982480879366998724564989139223479638211330615126531382500165813146013432141921007519873244667685688697608499623716290964167431105223792820657648271245518032937618686796051937676140461205559570203250915064523984028047382459761480695552369884522932828834483008799, 100))

4096
True


In [24]:
#napisz funkcję generującą liczbę pierwszą 
def generatePrime(keysize, cnt=0):
    #napisz swój kod tutaj
    
    for i in range(2**(keysize-1), 2**keysize):
        if isPrime3(i, 30):
            if cnt == 0:
                return i
            else: cnt -= 1
    
    return 0

generatePrime(16)

32771

## Zastanów się 
1. Znajdź zestawienie liczb silnie pseudopierwszych i sprawdź działanie (pojedynczego!) testy R-M dla takich liczb
2. Ile testów R-M należy przeprowadzić aby osiągnąć bezpieczny poziom pewności testu

### 2. Obliczamy składniki kluczy 
1. Wybieramy dwie duże liczby pierwsze $p$ i $q$
2. Pierwszym składnikiem klucza jest moduł $n$ $n=p \times q$ 
3. Poszukujemy wykładnika publicznego $e$, który jest względnie pierwszy z $(p-1)\cdot (q-1)$ (czasami używane jest w miejscu pojęcie tocjentu lub funkcji Eulera: $\phi(n) = \phi(p)\cdot \phi(q) = (p − 1)·(q − 1)$)
4. Poszukujemy wykładnika prywatnego $d$, które jest odwrotnością $e\ (mod\ (p-1)\cdot (q-1))$: $de \equiv 1  (mod\ (p-1)\cdot (q-1))$
5. Kluczem publiczny jest para $(n, e)$, kluczem prywatnym jest para $(n, d)$.

## Zadanie 

1. Napisz funkcję generującą klucze RSA o ustalonym rozmiarze

In [57]:
### import random, sys, os

def generateKey(keySize):
    
    p, q = generatePrime(keySize//2), generatePrime(keySize//2, 2)
    print(p, q)
    n = p*q
    eu = (p-1)*(q-1)
    e = 0
    for i in range(3, eu):
        if gcd(eu, i) == 1:
            e = i
            break
    print("e:", e)
    if e == 0:
        print("Error: Cannot find e!")
        return
    
    def dVal(k,eu,e):
        return ((1+(k*eu))//e)
    d = 0
    for i in range(1000):
        if (dVal(i, eu, e)*e)%eu == 1:
            d = dVal(i, eu, e)
            break
    print("d:", d)
    if d == 0:
        print("Error: Cannot find d!")
        return
    
    return ((n, e), (n, d))

pub,priv = generateKey(16)
print(pub,priv)

131 139
e: 7
d: 2563
(18209, 7) (18209, 2563)


## Zadanie 

Napisz funkcje implementujące szyfrowanie i deszyfrowanie RSA (tzw. podręcznikowe)

### Szyfrowanie RSA 
Operacja szyfrowania: $c=m^e (mod\ n)$

In [63]:
def encrypt(message, modulus, exp, key):
    
    pub,priv = key
    
    def encrypt_letter(l, pub):
        e = pub[1]
        txt = 1
        while e != 0:
            txt = (txt * l)%pub[0]
            e -= 1
        return txt
    message_encrypted = ""
    
    for l in message:
        message_encrypted += (chr(encrypt_letter(ord(l), pub)))
    
    return message_encrypted

encrypt("ala", 100, 10, (pub, priv))


'㚭ໞ㚭'

### Deszyfrowanie RSA 
Operacja szyfrowanie $m = c^d (mod\ n)$

In [64]:
def decrypt(message_encrypted, modulus, exp, key):
  
    pub, priv = key
    def decrypt_letter(l, priv):
        d = priv[1]
        txt = 1
        while d != 0:
            txt = (txt * l) % priv[0]
            d -= 1
        return txt
    
    message_decrypted = ""
    
    for l in message_encrypted:
        message_decrypted += chr(decrypt_letter(ord(l), priv))

    return message_decrypted

In [68]:
text = "ala ma kota"
print(text)
mess = encrypt(text, 100, 10, (pub, priv))
print(mess)
mess_out = decrypt(mess, 100, 10, (pub, priv))
print(mess_out)

ala ma kota
㚭ໞ㚭⪌ㆯ㚭⪌ዏ↖ⴃ㚭
ala ma kota


## Zastanów się
1. Sprawdź działanie powyższej implementacji dla różnych wielkości klucza (podawane podczas generowania kluczy). 
2. Poszukaj informacji o trybie podręcznikowym RSA (*textbook RSA encryption*). Na czym polega? Jakie są jego wady i zalety? 


## Zadanie (dla chętnych)
1. Zademonstruj deformowalność podręcznikowego szyfrowania RSA
2. Zaimplementuj dopełnienie OAEP (_Optimal Assymmetric Encryption Padding_)
